This network is to test code done for Project 2. 
Go down to the 'full prediction' heading to check out the model in action!

In [1]:
import torch
import math
import prologue as prologue
import framework


torch.set_grad_enabled(False) #By the end we should have this set to False

In [2]:
######################################################################
######################### DATA things ################################
######################################################################
def generate_disc_set(nb): # This can be kept the same
    input = torch.empty(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).long()
    return input, target

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

# center the data:
mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

train_target = train_target.float()
test_target = test_target.float()

In [3]:
# Network sugested in the miniproj handout:
model = framework.Sequential((framework.Linear(2, 25), 
                              framework.Tanh(), 
                              framework.Linear(25, 25), 
                              framework.Tanh(),
                              framework.Linear(25, 25), 
                              framework.Tanh(),
                              framework.Linear(25, 1)))
                             
loss = framework.MSE()

In [4]:
model.train(train_input, train_target, nb_epochs=150, batch_size=50, learning_rate=0.0005, loss=loss)

epoch nb:  0 loss:  tensor([2270.3403])
epoch nb:  5 loss:  tensor([227.8352])
epoch nb:  10 loss:  tensor([167.1329])
epoch nb:  15 loss:  tensor([132.4987])
epoch nb:  20 loss:  tensor([111.2906])
epoch nb:  25 loss:  tensor([95.7027])
epoch nb:  30 loss:  tensor([85.4617])
epoch nb:  35 loss:  tensor([77.4297])
epoch nb:  40 loss:  tensor([71.2052])
epoch nb:  45 loss:  tensor([66.4516])
epoch nb:  50 loss:  tensor([62.7410])
epoch nb:  55 loss:  tensor([59.7449])
epoch nb:  60 loss:  tensor([57.2393])
epoch nb:  65 loss:  tensor([55.0807])
epoch nb:  70 loss:  tensor([53.1791])
epoch nb:  75 loss:  tensor([51.4757])
epoch nb:  80 loss:  tensor([49.9287])
epoch nb:  85 loss:  tensor([48.5049])
epoch nb:  90 loss:  tensor([47.1793])
epoch nb:  95 loss:  tensor([45.9365])
epoch nb:  100 loss:  tensor([44.7748])
epoch nb:  105 loss:  tensor([43.7016])
epoch nb:  110 loss:  tensor([42.7227])
epoch nb:  115 loss:  tensor([41.8316])
epoch nb:  120 loss:  tensor([41.0133])
epoch nb:  125 l

In [5]:
model.compute_nb_errors(train_input, train_target)
model.compute_nb_errors(test_input, test_target)

Accuracy on the data set is:  0.758
Accuracy on the data set is:  0.762


0.762

In [22]:
import torch
import math
from torch import nn

torch.set_grad_enabled(True) 

model_torch = torch.nn.Sequential(nn.Linear(2, 25), 
                              nn.Tanh(), 
                              nn.Linear(25, 25), 
                              nn.Tanh(),
                              nn.Linear(25, 25), 
                              nn.Tanh(),
                              nn.Linear(25, 1))

with torch.no_grad():
    for p in model_torch.parameters(): 
        #print('yahoo')
        #p.normal_(mean = 0, std = 1)
        
        dist = 1. / math.sqrt(25)
        p.uniform_(-dist, dist)
        #self.weights = torch.empty(dim_out, dim_in).uniform_(-dist, dist)
        #self.bias = torch.empty(dim_out).uniform_(-dist, dist)

In [23]:
def train(model, train_input, train_target, nb_epochs, batch_size, learning_rate, criterion=None):
    nb_samples = train_input.size(0)
    if train_input.size(0)%batch_size !=0:
        print("Batch size should divid length of training data.")
    for e in range(nb_epochs):
        loss_acc=0
        for b in range(0, nb_samples, batch_size):
            for i in range(batch_size):
                output = model.forward(train_input[b+i])[0]
                loss = criterion(output, train_target[b+i])
                loss_acc = loss_acc + loss.item()
            model.zero_grad()
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= learning_rate * p.grad
        if e%5 == 0:
            print('epoch nb: ', e, 'loss: ', loss_acc)

def compute_nb_errors(model, data_input, data_target, batch_size=1):
    tot_right = 0

    for b in range(0, data_input.size(0), batch_size):
        for i in range(batch_size):
            output = model(data_input[b+i])
            if output <0.0 and  data_target[b+i]==0: #this is specific to the toy dataset.
                tot_right+=1
            elif output >=0.0 and  data_target[b+i]==1:
                tot_right+=1

    print("Accuracy on the data set is: ", tot_right/data_input.size(0))
    return tot_right/data_input.size(0)

In [24]:
train(model_torch, train_input, train_target, nb_epochs=250, batch_size=50, learning_rate=0.0005, criterion = nn.MSELoss())

epoch nb:  0 loss:  508.7120053907769
epoch nb:  5 loss:  425.96650588011835
epoch nb:  10 loss:  368.76044107973576
epoch nb:  15 loss:  329.4614775143564
epoch nb:  20 loss:  302.70986783877015
epoch nb:  25 loss:  284.6933008655906
epoch nb:  30 loss:  272.69929188489914
epoch nb:  35 loss:  264.8118669837713
epoch nb:  40 loss:  259.69250524044037
epoch nb:  45 loss:  256.41771183907986
epoch nb:  50 loss:  254.35850177705288
epoch nb:  55 loss:  253.09171868860722
epoch nb:  60 loss:  252.33594851195812
epoch nb:  65 loss:  251.90609242022038
epoch nb:  70 loss:  251.6817117035389
epoch nb:  75 loss:  251.58523784577847
epoch nb:  80 loss:  251.56724989414215
epoch nb:  85 loss:  251.59663957357407
epoch nb:  90 loss:  251.6540762335062
epoch nb:  95 loss:  251.7277503311634
epoch nb:  100 loss:  251.81059500575066
epoch nb:  105 loss:  251.89849765598774
epoch nb:  110 loss:  251.98916421830654
epoch nb:  115 loss:  252.0813985168934
epoch nb:  120 loss:  252.1746461391449
epoch 

In [25]:
compute_nb_errors(model_torch, train_input, train_target)
compute_nb_errors(model_torch, test_input, test_target)

Accuracy on the data set is:  0.483
Accuracy on the data set is:  0.518


0.518

In [7]:
#############################
#############################
#############################
#############################
#############################

In [18]:
# define the model
model = framework.Sequential((framework.Linear(2, 5), 
                              framework.Tanh(), 
                              framework.Linear(5, 5), 
                              framework.Tanh(),
                              framework.Linear(5, 5), 
                              framework.Tanh(),
                              framework.Linear(5, 5), 
                              framework.Tanh(),
                              framework.Linear(5, 1)))

loss = framework.MSE()

# train it
model.train(train_input, train_target, nb_epochs=500, batch_size=50, learning_rate=0.0005, loss=loss)

epoch nb:  0 loss:  tensor([296.7399])
epoch nb:  5 loss:  tensor([252.8259])
epoch nb:  10 loss:  tensor([251.1049])
epoch nb:  15 loss:  tensor([249.9750])
epoch nb:  20 loss:  tensor([248.9857])
epoch nb:  25 loss:  tensor([247.9324])
epoch nb:  30 loss:  tensor([246.6447])
epoch nb:  35 loss:  tensor([244.9047])
epoch nb:  40 loss:  tensor([242.3646])
epoch nb:  45 loss:  tensor([238.4200])
epoch nb:  50 loss:  tensor([232.0292])
epoch nb:  55 loss:  tensor([221.6655])
epoch nb:  60 loss:  tensor([206.3883])
epoch nb:  65 loss:  tensor([189.1692])
epoch nb:  70 loss:  tensor([176.8733])
epoch nb:  75 loss:  tensor([171.2079])
epoch nb:  80 loss:  tensor([168.7868])
epoch nb:  85 loss:  tensor([167.4163])
epoch nb:  90 loss:  tensor([166.3062])
epoch nb:  95 loss:  tensor([165.1538])
epoch nb:  100 loss:  tensor([163.7706])
epoch nb:  105 loss:  tensor([161.9319])
epoch nb:  110 loss:  tensor([159.2716])
epoch nb:  115 loss:  tensor([155.1434])
epoch nb:  120 loss:  tensor([148.3601

In [19]:
model.compute_nb_errors(train_input, train_target, batch_size=1)
model.compute_nb_errors(test_input, test_target, batch_size=1)

Accuracy on the data set is:  0.705
Accuracy on the data set is:  0.697


0.697